In [1]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('netflix_titles.csv')
initial_count = df.shape[0]
print(df.shape)
print(initial_count)
df.head(10)

(8807, 12)
8807


,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...
5,s6,TV Show,Midnight Mass,Mike Flanagan,"Kate Siegel, Zach Gilford, Hamish Linklater, H...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"TV Dramas, TV Horror, TV Mysteries",The arrival of a charismatic young priest brin...
6,s7,Movie,My Little Pony: A New Generation,"Robert Cullen, José Luis Ucha","Vanessa Hudgens, Kimiko Glenn, James Marsden, ...",NaN,"September 24, 2021",2021,PG,91 min,Children & Family Movies,Equestria's divided. But a bright-eyed hero be...
7,s8,Movie,Sankofa,Haile Gerima,"Kofi Ghanaba, Oyafunmike Ogunlano, Alexandra D...","United States, Ghana, Burkina Faso, United Kin...","September 24, 2021",1993,TV-MA,125 min,"Dramas, Independent Movies, International Movies","On a photo shoot in Ghana, an American model s..."
8,s9,TV Show,The Great British Baking Show,Andy Devonshire,"Mel Giedroyc, Sue Perkins, Mary Berry, Paul Ho...",United Kingdom,"September 24, 2021",2021,TV-14,9 Seasons,"British TV Shows, Reality TV",A talented batch of amateur bakers face off in...
9,s10,Movie,The Starling,Theodore Melfi,"Melissa McCarthy, Chris O'Dowd, Kevin Kline, T...",United States,"September 24, 2021",2021,PG-13,104 min,"Comedies, Dramas",A woman adjusting to life after a loss contend...


In [4]:
df = df.drop_duplicates()
df.head(10)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...
5,s6,TV Show,Midnight Mass,Mike Flanagan,"Kate Siegel, Zach Gilford, Hamish Linklater, H...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"TV Dramas, TV Horror, TV Mysteries",The arrival of a charismatic young priest brin...
6,s7,Movie,My Little Pony: A New Generation,"Robert Cullen, José Luis Ucha","Vanessa Hudgens, Kimiko Glenn, James Marsden, ...",NaN,"September 24, 2021",2021,PG,91 min,Children & Family Movies,Equestria's divided. But a bright-eyed hero be...
7,s8,Movie,Sankofa,Haile Gerima,"Kofi Ghanaba, Oyafunmike Ogunlano, Alexandra D...","United States, Ghana, Burkina Faso, United Kin...","September 24, 2021",1993,TV-MA,125 min,"Dramas, Independent Movies, International Movies","On a photo shoot in Ghana, an American model s..."
8,s9,TV Show,The Great British Baking Show,Andy Devonshire,"Mel Giedroyc, Sue Perkins, Mary Berry, Paul Ho...",United Kingdom,"September 24, 2021",2021,TV-14,9 Seasons,"British TV Shows, Reality TV",A talented batch of amateur bakers face off in...
9,s10,Movie,The Starling,Theodore Melfi,"Melissa McCarthy, Chris O'Dowd, Kevin Kline, T...",United States,"September 24, 2021",2021,PG-13,104 min,"Comedies, Dramas",A woman adjusting to life after a loss contend...


In [5]:
count_after_drop_duplicates = df.shape[0]
count_drop = initial_count - count_after_drop_duplicates
print("Number of rows dropped: ", count_drop)
print(df.shape)
df.isnull().sum()

Number of rows dropped:  0
(8807, 12)


show_id            0
type               0
title              0
director        2634
cast             825
country          831
date_added        10
release_year       0
rating             4
duration           3
listed_in          0
description        0
dtype: int64

In [6]:
df = df.dropna(subset=['title', 'type']).reset_index(drop=True) #More logical
# df = df.dropna().reset_index(drop=True) #Not logical since dropping other coloumns than title and type makes no sense
print(df.shape)
count_after_dropping_na = df.shape[0]
count_drop_na = count_after_drop_duplicates - count_after_dropping_na
print("Number of rows dropped (NA): ", count_drop_na)
df.head(10)

(8807, 12)
Number of rows dropped (NA):  0


,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...
5,s6,TV Show,Midnight Mass,Mike Flanagan,"Kate Siegel, Zach Gilford, Hamish Linklater, H...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"TV Dramas, TV Horror, TV Mysteries",The arrival of a charismatic young priest brin...
6,s7,Movie,My Little Pony: A New Generation,"Robert Cullen, José Luis Ucha","Vanessa Hudgens, Kimiko Glenn, James Marsden, ...",NaN,"September 24, 2021",2021,PG,91 min,Children & Family Movies,Equestria's divided. But a bright-eyed hero be...
7,s8,Movie,Sankofa,Haile Gerima,"Kofi Ghanaba, Oyafunmike Ogunlano, Alexandra D...","United States, Ghana, Burkina Faso, United Kin...","September 24, 2021",1993,TV-MA,125 min,"Dramas, Independent Movies, International Movies","On a photo shoot in Ghana, an American model s..."
8,s9,TV Show,The Great British Baking Show,Andy Devonshire,"Mel Giedroyc, Sue Perkins, Mary Berry, Paul Ho...",United Kingdom,"September 24, 2021",2021,TV-14,9 Seasons,"British TV Shows, Reality TV",A talented batch of amateur bakers face off in...
9,s10,Movie,The Starling,Theodore Melfi,"Melissa McCarthy, Chris O'Dowd, Kevin Kline, T...",United States,"September 24, 2021",2021,PG-13,104 min,"Comedies, Dramas",A woman adjusting to life after a loss contend...


In [7]:
text_cols = ['country', 'type', 'director', 'cast', 'listed_in']
for col in text_cols:
    if col in df.columns:
        df[col] = df[col].astype(str).str.strip().replace('nan', np.nan)
        df[col] = df[col].where(df[col].isna(), df[col].str.lower())
df.head(10)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,movie,Dick Johnson Is Dead,kirsten johnson,NaN,united states,"September 25, 2021",2020,PG-13,90 min,documentaries,"As her father nears the end of his life, filmm..."
1,s2,tv show,Blood & Water,NaN,"ama qamata, khosi ngema, gail mabalane, thaban...",south africa,"September 24, 2021",2021,TV-MA,2 Seasons,"international tv shows, tv dramas, tv mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,tv show,Ganglands,julien leclercq,"sami bouajila, tracy gotoas, samuel jouy, nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"crime tv shows, international tv shows, tv act...",To protect his family from a powerful drug lor...
3,s4,tv show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"docuseries, reality tv","Feuds, flirtations and toilet talk go down amo..."
4,s5,tv show,Kota Factory,NaN,"mayur more, jitendra kumar, ranjan raj, alam k...",india,"September 24, 2021",2021,TV-MA,2 Seasons,"international tv shows, romantic tv shows, tv ...",In a city of coaching centers known to train I...
5,s6,tv show,Midnight Mass,mike flanagan,"kate siegel, zach gilford, hamish linklater, h...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"tv dramas, tv horror, tv mysteries",The arrival of a charismatic young priest brin...
6,s7,movie,My Little Pony: A New Generation,"robert cullen, josé luis ucha","vanessa hudgens, kimiko glenn, james marsden, ...",NaN,"September 24, 2021",2021,PG,91 min,children & family movies,Equestria's divided. But a bright-eyed hero be...
7,s8,movie,Sankofa,haile gerima,"kofi ghanaba, oyafunmike ogunlano, alexandra d...","united states, ghana, burkina faso, united kin...","September 24, 2021",1993,TV-MA,125 min,"dramas, independent movies, international movies","On a photo shoot in Ghana, an American model s..."
8,s9,tv show,The Great British Baking Show,andy devonshire,"mel giedroyc, sue perkins, mary berry, paul ho...",united kingdom,"September 24, 2021",2021,TV-14,9 Seasons,"british tv shows, reality tv",A talented batch of amateur bakers face off in...
9,s10,movie,The Starling,theodore melfi,"melissa mccarthy, chris o'dowd, kevin kline, t...",united states,"September 24, 2021",2021,PG-13,104 min,"comedies, dramas",A woman adjusting to life after a loss contend...


In [8]:
if 'date_added' in df.columns:
    from dateutil import parser
    import warnings
    warnings.filterwarnings('ignore')
    
    def parse_date_flexible(date_str):
        if pd.isna(date_str):
            return None
        try:
            return parser.parse(str(date_str))
        except:
            try:
                return pd.to_datetime(date_str, format='%B %d, %Y')
            except:
                return None
    
    df['date_added'] = df['date_added'].apply(parse_date_flexible)
    
    df['date_added'] = pd.to_datetime(df['date_added'], errors='coerce').dt.strftime('%d-%m-%Y')

if 'release_year' in df.columns:
    df['release_year'] = pd.to_numeric(df['release_year'], errors='coerce').astype('Int64')
    
if 'rating' in df.columns:
    df['rating'] = df['rating'].astype('category')
    
if 'type' in df.columns:
    df['type'] = df['type'].astype('category')
    
df.head(10)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,movie,Dick Johnson Is Dead,kirsten johnson,NaN,united states,25-09-2021,2020,PG-13,90 min,documentaries,"As her father nears the end of his life, filmm..."
1,s2,tv show,Blood & Water,NaN,"ama qamata, khosi ngema, gail mabalane, thaban...",south africa,24-09-2021,2021,TV-MA,2 Seasons,"international tv shows, tv dramas, tv mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,tv show,Ganglands,julien leclercq,"sami bouajila, tracy gotoas, samuel jouy, nabi...",NaN,24-09-2021,2021,TV-MA,1 Season,"crime tv shows, international tv shows, tv act...",To protect his family from a powerful drug lor...
3,s4,tv show,Jailbirds New Orleans,NaN,NaN,NaN,24-09-2021,2021,TV-MA,1 Season,"docuseries, reality tv","Feuds, flirtations and toilet talk go down amo..."
4,s5,tv show,Kota Factory,NaN,"mayur more, jitendra kumar, ranjan raj, alam k...",india,24-09-2021,2021,TV-MA,2 Seasons,"international tv shows, romantic tv shows, tv ...",In a city of coaching centers known to train I...
5,s6,tv show,Midnight Mass,mike flanagan,"kate siegel, zach gilford, hamish linklater, h...",NaN,24-09-2021,2021,TV-MA,1 Season,"tv dramas, tv horror, tv mysteries",The arrival of a charismatic young priest brin...
6,s7,movie,My Little Pony: A New Generation,"robert cullen, josé luis ucha","vanessa hudgens, kimiko glenn, james marsden, ...",NaN,24-09-2021,2021,PG,91 min,children & family movies,Equestria's divided. But a bright-eyed hero be...
7,s8,movie,Sankofa,haile gerima,"kofi ghanaba, oyafunmike ogunlano, alexandra d...","united states, ghana, burkina faso, united kin...",24-09-2021,1993,TV-MA,125 min,"dramas, independent movies, international movies","On a photo shoot in Ghana, an American model s..."
8,s9,tv show,The Great British Baking Show,andy devonshire,"mel giedroyc, sue perkins, mary berry, paul ho...",united kingdom,24-09-2021,2021,TV-14,9 Seasons,"british tv shows, reality tv",A talented batch of amateur bakers face off in...
9,s10,movie,The Starling,theodore melfi,"melissa mccarthy, chris o'dowd, kevin kline, t...",united states,24-09-2021,2021,PG-13,104 min,"comedies, dramas",A woman adjusting to life after a loss contend...


In [9]:
df.columns = [col.strip().lower().replace(' ', '_') for col in df.columns]
df.head(10)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,movie,Dick Johnson Is Dead,kirsten johnson,NaN,united states,25-09-2021,2020,PG-13,90 min,documentaries,"As her father nears the end of his life, filmm..."
1,s2,tv show,Blood & Water,NaN,"ama qamata, khosi ngema, gail mabalane, thaban...",south africa,24-09-2021,2021,TV-MA,2 Seasons,"international tv shows, tv dramas, tv mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,tv show,Ganglands,julien leclercq,"sami bouajila, tracy gotoas, samuel jouy, nabi...",NaN,24-09-2021,2021,TV-MA,1 Season,"crime tv shows, international tv shows, tv act...",To protect his family from a powerful drug lor...
3,s4,tv show,Jailbirds New Orleans,NaN,NaN,NaN,24-09-2021,2021,TV-MA,1 Season,"docuseries, reality tv","Feuds, flirtations and toilet talk go down amo..."
4,s5,tv show,Kota Factory,NaN,"mayur more, jitendra kumar, ranjan raj, alam k...",india,24-09-2021,2021,TV-MA,2 Seasons,"international tv shows, romantic tv shows, tv ...",In a city of coaching centers known to train I...
5,s6,tv show,Midnight Mass,mike flanagan,"kate siegel, zach gilford, hamish linklater, h...",NaN,24-09-2021,2021,TV-MA,1 Season,"tv dramas, tv horror, tv mysteries",The arrival of a charismatic young priest brin...
6,s7,movie,My Little Pony: A New Generation,"robert cullen, josé luis ucha","vanessa hudgens, kimiko glenn, james marsden, ...",NaN,24-09-2021,2021,PG,91 min,children & family movies,Equestria's divided. But a bright-eyed hero be...
7,s8,movie,Sankofa,haile gerima,"kofi ghanaba, oyafunmike ogunlano, alexandra d...","united states, ghana, burkina faso, united kin...",24-09-2021,1993,TV-MA,125 min,"dramas, independent movies, international movies","On a photo shoot in Ghana, an American model s..."
8,s9,tv show,The Great British Baking Show,andy devonshire,"mel giedroyc, sue perkins, mary berry, paul ho...",united kingdom,24-09-2021,2021,TV-14,9 Seasons,"british tv shows, reality tv",A talented batch of amateur bakers face off in...
9,s10,movie,The Starling,theodore melfi,"melissa mccarthy, chris o'dowd, kevin kline, t...",united states,24-09-2021,2021,PG-13,104 min,"comedies, dramas",A woman adjusting to life after a loss contend...


In [10]:
print('Final shape:', df.shape)
print('\nNull counts after cleaning:\n', df.isnull().sum(), '\n')
print(df.dtypes)
df.to_csv('cleaned_netflix_titles.csv', index=False)
print('Saved cleaned dataset to cleaned_netflix_titles.csv')
df.head(10)

Final shape: (8807, 12)

Null counts after cleaning:
 show_id            0
type               0
title              0
director        2634
cast             825
country          831
date_added        10
release_year       0
rating             4
duration           3
listed_in          0
description        0
dtype: int64 

show_id           object
type            category
title             object
director          object
cast              object
country           object
date_added        object
release_year       Int64
rating          category
duration          object
listed_in         object
description       object
dtype: object
Saved cleaned dataset to cleaned_netflix_titles.csv


,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,movie,Dick Johnson Is Dead,kirsten johnson,NaN,united states,25-09-2021,2020,PG-13,90 min,documentaries,"As her father nears the end of his life, filmm..."
1,s2,tv show,Blood & Water,NaN,"ama qamata, khosi ngema, gail mabalane, thaban...",south africa,24-09-2021,2021,TV-MA,2 Seasons,"international tv shows, tv dramas, tv mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,tv show,Ganglands,julien leclercq,"sami bouajila, tracy gotoas, samuel jouy, nabi...",NaN,24-09-2021,2021,TV-MA,1 Season,"crime tv shows, international tv shows, tv act...",To protect his family from a powerful drug lor...
3,s4,tv show,Jailbirds New Orleans,NaN,NaN,NaN,24-09-2021,2021,TV-MA,1 Season,"docuseries, reality tv","Feuds, flirtations and toilet talk go down amo..."
4,s5,tv show,Kota Factory,NaN,"mayur more, jitendra kumar, ranjan raj, alam k...",india,24-09-2021,2021,TV-MA,2 Seasons,"international tv shows, romantic tv shows, tv ...",In a city of coaching centers known to train I...
5,s6,tv show,Midnight Mass,mike flanagan,"kate siegel, zach gilford, hamish linklater, h...",NaN,24-09-2021,2021,TV-MA,1 Season,"tv dramas, tv horror, tv mysteries",The arrival of a charismatic young priest brin...
6,s7,movie,My Little Pony: A New Generation,"robert cullen, josé luis ucha","vanessa hudgens, kimiko glenn, james marsden, ...",NaN,24-09-2021,2021,PG,91 min,children & family movies,Equestria's divided. But a bright-eyed hero be...
7,s8,movie,Sankofa,haile gerima,"kofi ghanaba, oyafunmike ogunlano, alexandra d...","united states, ghana, burkina faso, united kin...",24-09-2021,1993,TV-MA,125 min,"dramas, independent movies, international movies","On a photo shoot in Ghana, an American model s..."
8,s9,tv show,The Great British Baking Show,andy devonshire,"mel giedroyc, sue perkins, mary berry, paul ho...",united kingdom,24-09-2021,2021,TV-14,9 Seasons,"british tv shows, reality tv",A talented batch of amateur bakers face off in...
9,s10,movie,The Starling,theodore melfi,"melissa mccarthy, chris o'dowd, kevin kline, t...",united states,24-09-2021,2021,PG-13,104 min,"comedies, dramas",A woman adjusting to life after a loss contend...
